In [36]:
import torch

In [37]:
video_dir = 'datasets/mp4'
json_dir = 'datasets/data/'
label_dir = 'datasets/cfg/'
list_file = 'datafiles.txt'

In [38]:
import numpy as np
import pandas as pd
import os
with open(list_file,'w') as datafiles:
    filelist = os.listdir(video_dir)
    for i in filelist:
        datafiles.write(i[:-4]+',')

In [39]:
filelist = []
with open(list_file,'r') as file:
    line = file.readline()
    filelist = line.split(',')

filelist

['dataset-1_bow_active_001dataset-1_bow_angry_001dataset-1_bow_childish_001dataset-1_bow_chimpira_001dataset-1_bow_feminine_001dataset-1_bow_giant_001dataset-1_bow_happy_001dataset-1_bow_masculinity_001dataset-1_bow_musical_001dataset-1_bow_normal_001dataset-1_bow_not-confident_001dataset-1_bow_old_001dataset-1_bow_proud_001dataset-1_bow_sad_001dataset-1_bow_tired_001dataset-1_byebye_active_001dataset-1_byebye_angry_001dataset-1_byebye_childish_001dataset-1_byebye_chimpira_001dataset-1_byebye_feminine_001dataset-1_byebye_giant_001dataset-1_byebye_happy_001dataset-1_byebye_masculinity_001dataset-1_byebye_musical_001dataset-1_byebye_normal_001dataset-1_byebye_not-confident_001dataset-1_byebye_old_001dataset-1_byebye_proud_001dataset-1_byebye_sad_001dataset-1_byebye_tired_001dataset-1_bye_active_001dataset-1_bye_angry_001dataset-1_bye_childish_001dataset-1_bye_chimpira_001dataset-1_bye_feminine_001dataset-1_bye_giant_001dataset-1_bye_happy_001dataset-1_bye_masculinity_001dataset-1_bye_mus